In [18]:
import pandas as pd 
import os 
import numpy as np 

In [7]:
centro = '8811417' #input("Introduza o centro: ")

In [8]:

folder_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/Dados_centros")

files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]


combined_df = pd.DataFrame()


for file in files:
    file_path = os.path.join(folder_path, file)
    
    
    df = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    
    
    combined_df = pd.concat([combined_df, df], ignore_index=True)



In [9]:
# -- limpeza do dataset 
df = combined_df
df.columns=[x.lower().strip() for x in df.columns]
df.drop_duplicates()
df['centro'] = df['centro'].astype(str).str.replace('.0',"")
df['centro'] = df['centro'].str.strip()
df['giro'] = df['giro'].str.replace(' ', '').str.strip().str[:5]
df['cp'] = df['cp'].str.replace(' ', '').str.strip()
df['longitude'] = df['longitude'].str.replace('\n', '').str.strip()
df=df[df.latitude != '0']
df=df[df.latitude != '0.0']
df = df[df['latitude'] != '']
df=df[df.cod_t_even == 'I']
df = df[df['cp'].str.len() >= 8]

In [10]:
df=df[df['centro']==centro]

In [11]:

giro_unique_ocorrencias = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_dias')

loptica_unique_ocorrencias = df.groupby('giro')['loptica'].nunique().reset_index(name='N_objetos')

giros_pit_stop = pd.merge(giro_unique_ocorrencias, loptica_unique_ocorrencias, left_on='giro', right_on='giro', how='inner')

In [12]:
import pandas as pd

df['cp4_prestador'] = df['giro'].str[:4]
df['cp4_real'] = df['cp'].str[:4]

contagem_linhas = df.groupby(['cp4_prestador', 'cp4_real']).size().reset_index(name='n_objetos')

soma_total_cp4_prestador = contagem_linhas.groupby('cp4_prestador')['n_objetos'].sum()

contagem_linhas = pd.merge(contagem_linhas, soma_total_cp4_prestador, on='cp4_prestador', suffixes=('', '_total'))

contagem_filtrada = contagem_linhas[contagem_linhas['n_objetos'] > contagem_linhas['n_objetos_total'] / 10]

contagem_filtrada = contagem_filtrada.drop(columns=['n_objetos_total'])

contagem_filtrada


,cp4_prestador,cp4_real,n_objetos
0,3700,3700,5764
7,3720,3720,7900
12,3730,3730,2952
15,3750,3750,4628
23,3770,3770,2680
30,3800,3800,6654
33,3800,3810,7864
41,3830,3830,4663
48,3840,3840,2542
53,3850,3850,3438


In [17]:
import os
import pandas as pd

downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")

file_path = os.path.join(downloads_path, f'{centro}.xlsx')

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    
    giros_pit_stop.to_excel(writer, sheet_name='giros_pit_stop', index=False)
    
    contagem_filtrada.to_excel(writer, sheet_name='áreas de atuação', index=False)

print(f"File saved at: {file_path}")


File saved at: C:\Users\1420844\Downloads\8811417.xlsx
